## LSTM

In [60]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from tensorflow.keras.layers import Flatten,LSTM, Dense, Flatten, Embedding
import keras
from keras import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.text import Tokenizer
from keras.initializers import glorot_uniform
from sklearn import model_selection

In [51]:
%cd "/Users/jasper/Documents/python/Neual_net/data"
with open('train.csv', 'r') as file:
   text = file.readlines()

text[0]

/Users/jasper/Documents/python/Neual_net/data


'"2","Stuning even for the non-gamer","This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^"\n'

In [52]:
# create dataframe
Xtrain = pd.DataFrame()
word = []
label = []

for n in text:
    n = n.split(',')
    label.append(1) if n[0] == '"2"' else label.append(0)
    word.append(' '.join(n[1:]))
Xtrain['review'] = word; Xtrain['label'] = label
Xtrain.head(10)

,review,label
0,"""Stuning even for the non-gamer"" ""This sound t...",1
1,"""The best soundtrack ever to anything."" ""I'm r...",1
2,"""Amazing!"" ""This soundtrack is my favorite mus...",1
3,"""Excellent Soundtrack"" ""I truly like this soun...",1
4,"""Remember Pull Your Jaw Off The Floor After H...",1
5,"""an absolute masterpiece"" ""I am quite sure any...",1
6,"""Buyer beware"" ""This is a self-published book ...",0
7,"""Glorious story"" ""I loved Whisper of the wicke...",1
8,"""A FIVE STAR BOOK"" ""I just finished reading Wh...",1
9,"""Whispers of the Wicked Saints"" ""This was a ea...",1


In [32]:
Xtrain['label'].value_counts()

1    1800000
0    1800000
Name: label, dtype: int64

In [53]:
#use only 2% of data to avoid overloading your system

_, Xset, __, yset = model_selection.train_test_split(Xtrain['review'],
                                                    Xtrain['label'],
                                                    test_size = 0.01)

In [54]:
# remove punctuation, convert upper to lower case, and remove single character

def preparation(tex):
    out_tex = re.sub('[^a-zA-Z]', ' ', tex)
    out_tex = ''.join(list(map(lambda x: x.lower(),out_tex)))
    out_tex = re.sub(r'\s+[a-zA-Z]\s', ' ',out_tex) #r'', do not process escape sequences 
    return out_tex

In [55]:
text = []
for review in list(Xset):
    text.append(preparation(review))
Xtrain = pd.DataFrame()
Xtrain['review'] = text
Xtrain['label'] = list(yset)
Xtrain.head()

,review,label
0,outback station read outback the first in...,1
1,top ranking reggae river niger believe fo...,1
2,don bore the jukebox this cd starts off g...,0
3,rubbish cant believe how many good reviews th...,0
4,what disappointment stopped watching this mov...,0


In [56]:
# convert to arrays
Xtrain, Xtest, ytrain, ytest = model_selection.train_test_split(Xtrain['review'],
                                                               Xtrain['label'],
                                                               test_size = 0.3)
Xtrain = np.array(Xtrain.values.tolist())
Xtest = np.array(Xtest.values.tolist())
ytrain = np.array(ytrain.values.tolist())
ytest = np.array(ytest.values.tolist())

In [57]:
# tokenizer
tok = Tokenizer()
tok.fit_on_texts(Xtrain)
word_index = tok.word_index
total_size = len(word_index)+1
print(total_size)

53835


In [58]:
Xtrain = tok.texts_to_sequences(Xtrain)
Xtest = tok.texts_to_sequences(Xtest)

# add padding to ensure same length
max_length = 100
Xtrain = pad_sequences(Xtrain, padding = 'post', maxlen = max_length)
Xtest = pad_sequences(Xtest, padding = 'post', maxlen = max_length)

In [69]:
# Create model
model = tf.keras.Sequential()
model.add(Embedding(total_size, 20, input_length=max_length))
model.add(LSTM(32,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 20)           1076700   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                6784      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 1,083,517
Trainable params: 1,083,517
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [71]:
model.fit(Xtrain, ytrain, batch_size=128, epochs=5, validation_data=(Xtest, ytest))


Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
197/197 [==============================] - ETA: 0s - loss: 0.6776 - accuracy: 0.5464WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fd7c996c8c0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and

In [94]:
prob = model.predict(Xtest)
pred = tf.round(prob)
df = pd.DataFrame({'prediction':tf.squeeze(pred,axis = -1).numpy(),'true':ytest.to_numpy()})
df.head()

,prediction,true
0,1.0,1
1,0.0,1
2,1.0,1
3,1.0,1
4,0.0,0


In [101]:
df['acc'] = df.apply(lambda x: 1 if x['prediction'] == x['true'] else 0, axis=1)

In [103]:
df.acc.sum()/len(df) # same from the

0.8241666666666667